# Create Users

## Présentation
L'objectif de ce script est d'agréger les données relatives aux utilisateurs qui ont envoyé un tweet et de les stocker en base de données afin qu'elles puissent être utilisées ultérieurement pour d'autres analyses, telles que la classification ou le clustering d'utilisateurs en fonction de s'ils sont atypiques ou non.

Que fait-il ?
1. Récupère le dernier tweet qui a été envoyé. Les tweets étant figés, cette récupération permet d'avoir des durées cohérentes, pas trop importantes.
2. On agrège les données des utilisateurs en fonctions des tweets qu'ils ont envoyés.
3. Calcul d'attributs supplémentaire qui pourront être utilisé pour classifier et créer des clusters d'utilisateur


Pour réaliser la classification et le clustering, nous avons décidé de définir un utilisateur de twitter par :
- S'il s'agit ou non d'un compte vérifié
- Le nombre total de tweet envoyé
- ratio friend/followers
- Le nombre total de tweet aimés par l'utilisateur
- l'âge du compte
- Le nombre moyen de RT qu'il possède sur ces tweets
- Fréquence d'envois de message par jour
- Ratio du nombre de hastag par tweet
- Degré d'agressivité
- ça visibilité
- le ratio du nombre de messages sensible qu'il a envoyé
- ...

## Le code
On importe les différentes bibliothèques nécessaires.

In [ ]:
import pymongo
import Levenshtein
import pandas as pd
from sklearn.preprocessing import StandardScaler

Connexion à la base de données MongoDB et plus précisément, aux deux collections utiles pour la classification et clustering:
- ``tweets`` dédiée à stocker les différents tweets initiaux
- ``users`` dédiée à stocker les différents utilisateurs de Twitter

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
tweet_collection = db["tweets"]
user_collection = db["users"]

On supprime toute la collection pour supprimer par la même occasion les données qu'elle contient.

In [ ]:
user_collection.drop()

Récupération du dernier tweet de la base de données pour pouvoir par la suite estimer l'âge d'un compte utilisateur à partir de cette date et non celle d'aujourd'hui pour éviter d'avoir des valeurs énormes, car les tweets sont figés.

In [ ]:
%%time
pipeline = [
    {
        "$sort" : {
            "created_date" : -1
        }
    },
    {
        "$limit": 1
    }
]
last_tweet_published = tweet_collection.aggregate(pipeline)
last_tweet_published = list(last_tweet_published)[0]
last_tweet_published

Création de variables générales aux tweets : nombre moyens de hashtage et de mentions pour évaluer le coup dans le cadre de la visibilité

In [ ]:
'''
pipeline_hash_men= [
    {
        "$group": {
            "_id": "null",
            "mentions": {
                "$avg":  {
                    "$avg":{
                        "$map": {
                        "input": "$entities.user_mentions",
                        "as": "mention",
                        "in": {"$strLenCP": "$$mention.screen_name"}
                        }
                    }
                }
            },
            "hash": {
                "$avg": {
                    "$avg":{
                        "$map": {
                        "input": "$entities.hashtags",
                        "as": "hashtags",
                        "in": {"$strLenCP": "$$hashtags.text"}
                        }
                    }
                }
            }
        }
    }
]
res = list(tweet_collection.aggregate(pipeline_hash_men))
res
avg_mention = res[0]['mentions']
avg_hashtag = res[0]['hash']
'''
avg_mention = 10.581222288344232
avg_hashtag = 8.036196913497982

Isoler et regrouper les données des utilisateurs de Twitter pour ensuite calculer des attributs supplémentaires et les stocker en base de données.

In [ ]:
%%time
users = tweet_collection.aggregate([
    {
        "$group": {
            "_id": "$user.id",
            "tweets": { "$push": "$$ROOT" },
            "hashtag_frequency":{
                "$avg":{
                    "$size":"$entities.hashtags"
                }
            },
        }
    },
    {
        "$project": {
            "_id": 0,
            "user_id": "$_id",
            "tweets": 1,
            "hashtag_frequency": 1
        }
    },
    {
        "$sort" : {
            "tweets.created_date" : 1
        }
    },
    {
        "$addFields": {
            "verified": {
                "$cond": [
                    {'$eq': [{"$toString": {"$last": "$tweets.user.verified"}}, "true"]},
                    1,
                    0
                ]
            },
            "statuses_count": {"$last": "$tweets.user.statuses_count"},
            "favourites_count": {"$last": "$tweets.user.favourites_count"},
            "friends_count":{ "$last": "$tweets.user.friends_count" },
            "followers_count":{ "$last": "$tweets.user.followers_count" },
            "age_account": {
                "$divide": [
                  {
                    "$subtract": [last_tweet_published['created_date'], {"$last": "$tweets.user.created_at"}]
                  },
                  86400000 # nombre de millisecondes dans une journée
                ]
            },
            "last_tweet_published_id": {"$last": "$tweets._id"},
             "ratio_friends_followers": {
                "$cond": [
                { "$eq": [ { "$last": "$tweets.user.followers_count" }, 0 ] },
                -1,
                {
                    "$divide": [
                    { "$last": "$tweets.user.friends_count" },
                    { "$last": "$tweets.user.followers_count" }
                    ]
                }
                ]
            }
        }
    },
    {
        "$addFields": {
            "avg_tweet_length": {
                "$avg": {
                    "$map": {
                        "input": "$tweets",
                        "as": "tweet",
                        "in": { "$strLenCP": "$$tweet.text" }
                    }
                }
            },
            "tweet_frequency":{
                "$divide": [
                   "$statuses_count", "$age_account" 
                ]
            },
            "frequency_friend_hour":{
                "$divide":[
                  {
                    "$sum":{
                        "$add":[
                            { "$size": { "$ifNull": ["$tweets.entities.media", []] }},
                            { "$size":"$tweets.entities.urls"}
                        ]
                    }
                  },
                  {
                      "$multiply": [ "$age_account", 24 ] 
                  }
                ]
            },
            "nb_sensitive_tweets": {
                "$sum": {
                    "$map": {
                        "input": "$tweets",
                        "as": "tweet",
                        "in": { 
                            "$cond": [
                                {'$eq': [{"$toString": "$$tweet.possibly_sensitive"}, "true"]},
                                1,
                                0
                            ]
                        }
                    } 
                }
            }
        }
    },
    {
         "$addFields":{
              "Ap":{  #Dégré d'aggressivité
                "$divide":[
                  {
                      "$add": [
                          { "$divide":["$tweet_frequency",24] },
                          "$frequency_friend_hour"
                        ]
                  },
                  350
                ]
            },
            "visibility": {
                "$ifNull": [
                    {
                        "$divide": [
                            {
                                "$add": [
                                {
                                    "$multiply": [    
                                        avg_mention
                                        ,
                                        {
                                            "$avg": {
                                                "$map": {
                                                    "input": "$tweets.entities.user_mentions",
                                                    "as": "mention",
                                                    "in": { "$size": "$$mention" }
                                                }
                                            }
                                        }
                                    ]
                                },
                                {
                                    "$multiply": [
                                        avg_hashtag,
                                        {
                                            "$avg": {
                                                "$map": {
                                                    "input": "$tweets.entities.hashtags",
                                                    "as": "hashtag",
                                                    "in": { "$size": "$$hashtag" }
                                                }
                                            }
                                        }
                                    ]
                                }
                                ]
                            }, 140
                        ]
                    },
                    0
                ]
            },
            "ratio_sensitive_tweets": {
                "$cond": [
                    { "$ne": [{ "$size": "$tweets" }, 0] },
                    { 
                        "$divide": [
                            "$nb_sensitive_tweets", 
                            { "$size": "$tweets" }
                        ] 
                    },
                    -1
                ]
            },
            "ratio_punctuation_tweets": {
                "$avg": {
                    "$map": {
                        "input": "$tweets",
                        "as": "tweet",
                        "in": { "$divide": [
                            {
                                "$size": {
                                    "$regexFindAll": {
                                        "input": "$$tweet.text",
                                        "regex": r"[.,\/#!$%\^&\*;:{}=\-_`~()]"
                                    }
                                }
                            },
                            {"$strLenCP": "$$tweet.text"}
                        ] }
                    }
                }
            },
         }
    },
    {
        "$project": {
            "user_id": 1,
            "tweet_ids": "$tweets._id",
            "avg_tweet_length": 1,
            "tweet_frequency": 1,
            "last_tweet_published_id": 1,
            "verified": 1,
            "statuses_count": 1,
            "favourites_count": 1,
            "friends_count":1,
            "followers_count":1,
            "ratio_friends_followers": 1,
            "age_account": 1,
            "hashtag_frequency":1,
            "visibility": 1,
            "Ap":1,
            "ratio_sensitive_tweets": 1,
            "nb_sensitive_tweets": 1,
            "ratio_punctuation_tweets": 1,
            "_id": 0
        }
    }
])

user_collection.insert_many(list(users))

Nous avons décidé d'ajouter un nouvel attribut à nos utilisateurs.

La distance de Levenshtein, également connue sous le nom de distance d'édition, désigne la mesure de la différence entre deux chaînes de caractères en calculant le nombre minimum d'opérations nécessaires pour transformer l'une en l'autre. Les opérations possibles comprennent l'insertion, la suppression ou la substitution d'un caractère.

Plus la distance de Levenshtein entre deux chaînes de caractères est faible, plus elles sont similaires.

Nous allons utiliser cette distance de Levenshtein afin de mesurer la similarité des contenus des tweets publiés pour chaque utilisateur. Ainsi, si un utilisateur publie tout le temps le même tweets, cette distance sera d'autant plus faible. Cependant, s'il publie jamais les mêmes tweets, cette distance sera grande.

In [ ]:
def get_mean_levenshtein_distance(user):
    total_distance = 0
    num_pairs = 0
    for tweet1 in user['tweets']:
        for tweet2 in user['tweets']:
            if tweet1['id'] != tweet2['id']:
                total_distance += Levenshtein.distance(tweet1['text'], tweet2['text'])
                num_pairs += 1
    if num_pairs > 0:
        return total_distance / num_pairs
    else:
        return 0

Itérer sur tous les documents et mettre à jour chaque document avec le nouveau champ "avg_tweet_levenshtein_similarity"

In [ ]:
%%time
pipeline = [
    {
        '$lookup': {
            'from': 'tweets',
            'localField': 'tweet_ids',
            'foreignField': '_id',
            'as': 'tweets'
        }
    },
    {
        '$project': {
            'tweets.id': 1,
            'tweets.text': 1,
        }
    }
]

cursor = user_collection.aggregate(pipeline)

In [ ]:
%%time
for i, user in enumerate(cursor):
    distance = get_mean_levenshtein_distance(user)
    user_collection.update_one({'_id': user['_id']}, {'$set': {'avg_tweet_levenshtein_similarity': distance}})
    if i % 100_000 == 0:
        print(f'Processed {i} documents')

## Standardisation

La standardisation des données est une étape essentielle dans le processus de prétraitement des données. Elle vise à mettre les variables sur une échelle commune afin de faciliter leur comparaison et leur utilisation dans des modèles d'apprentissage automatique.

Nous nous assurons alors que toutes les variables sont traitées de manière équitable et cohérente, ce qui améliore la performance des modèles d'apprentissage automatique et facilite l'interprétation des résultats. Cela permet également d'éviter que certaines variables avec des valeurs plus élevées ne dominent les autres dans les calculs et les estimations. 

La standardisation consiste à centrer les données autour de zéro en soustrayant la moyenne de chaque variable et en divisant par l'écart-type. Cela permet d'éliminer les écarts de magnitude entre les différentes variables. 

Cela garantit par ailleurs que toutes les variables ont une moyenne de zéro et un écart-type de un.

Définition de la collection qui va contenir les utilisateurs normalisés.

In [ ]:
user_scaled_collection = db["users_scaled"]

On supprime toute la collection pour supprimer par la même occasion les données qu'elle contient. On réinitialise donc la collection.

In [ ]:
user_scaled_collection.drop()

Récupération de l'ensemble des utilisateurs

In [ ]:
users_mongo = list(user_collection.find({}))

On filtre les données que l'on souhaite standardisé

In [ ]:
users = pd.DataFrame(users_mongo)
data_to_scale = users.drop(columns=["_id","user_id","last_tweet_published_id","tweet_ids"])

In [ ]:
data_to_scale.describe()

Standardisation des utilisateurs

In [ ]:
scaler = StandardScaler()
users_scaled = scaler.fit_transform(data_to_scale)

attributs = data_to_scale.columns.tolist()
users_scaled_df = pd.DataFrame(users_scaled, columns=attributs)

users_scaled_with_ids = pd.concat([users_scaled_df, users['_id'], users['user_id']], axis=1)

In [ ]:
users_scaled_with_ids.describe()

Insertions des utilisateurs normalisés dans la collection

In [ ]:
users_scaled_with_ids = users_scaled_with_ids.to_dict(orient='records')
user_scaled_collection.insert_many(users_scaled_with_ids)

## Affichage

In [ ]:
pipeline = [
    {"$unwind": "$tweet_ids"},
    {"$group": {"_id": "$user_id", "tweet_ids_count": {"$sum": 1}}},
    {"$sort":{"tweet_ids_count":-1}}
]

# Execute the aggregation pipeline
result = user_collection.aggregate(pipeline)

# Print the results
for doc in result:
    print("user_id: {}, tweet_ids_count: {}".format(doc["_id"], doc["tweet_ids_count"]))